# Exploring the Cityscapes dataset

In this Jupyter Notebook we explore the Cityscapes dataset making use of the dataloader [cityscapesDataset](./datasets/cityscapes.py).

In [ ]:
!pip install cityscapesscripts;

## Imports
We make use of the [cityscapesscripts package](https://github.com/mcordts/cityscapesScripts) to download the files. It is necessary to provide a *username* and *password* to download the dataset. 

In [1]:
import os
import torch
import zipfile
import numpy as np

from importlib import reload
from os.path import join as pjoin
from torchvision import transforms
from cityscapesscripts.download import downloader 
from datasets.cityscapes import cityscapesDataset

In [ ]:
local_path = "./Cityscapes"
pckgs_names = ["gtFine_trainvaltest.zip","leftImg8bit_trainvaltest.zip"]
dir_names = ["gtFine", "leftImg8bit"]

### Download the dataset

In order to obtain the necessary credentials, visit the [Cityscapes dataset webpage](https://www.cityscapes-dataset.com/register/) and register. Once you have registered, execute the next cell to log in.

In [ ]:
session = downloader.login()

In [ ]:
# Download zip files
for pckg_name in pckgs_names:
    if not os.path.isfile(pckg_name):
        session = downloader.login()
        downloader.download_packages(session=session, package_names=[pckg_name], destination_path="./")

In [5]:
# Extract zip files 
for i, dir_name in enumerate(dir_names):
    if not os.path.isdir(pjoin(local_path, dir_name)):
        with zipfile.ZipFile(pckgs_names[i], "r") as zip_file:
            zip_file.extractall(local_path)

### Load the dataset

In [6]:
training_data = cityscapesDataset(local_path, split="train")
validation_data = cityscapesDataset(local_path, split="val")
test_data = cityscapesDataset(local_path, split="test")

Processing 5000 annotation files
Progress: 100.0 %    Annotations files processed
Annotations files processed


### Explore the dataset

In [ ]:
dataset_sizes = {
    'train': len(training_data),
    'val': len(validation_data),
    'test': len(test_data),
    'total': len(training_data) + len(validation_data) + len(test_data)
}

dataset_sizes

Both the image and its label are resized by default from (2048, 1024) to (512, 256).

In [ ]:
print("Image size = ", training_data[0][0].size())
print("Label size = ", training_data[0][1].size())

In [ ]:
_, label = training_data[5]
img, _ = training_data[5]

In [ ]:
transforms.ToPILImage()(img)

In [ ]:
training_data.decode_segmap(label.numpy(), plot=True)

#### Training label ids 
Labels **-1** and **255** shall be ignored.

In [ ]:
training_data.label_ids()

Check that resizing the label mask has not created invalid label ids

In [ ]:
valid_labels = set(training_data.label_ids())
all_valid = True
for i in range(50):
    j = np.random.randint(0, len(training_data))
    _, lbl = training_data[j]
    if not set(np.unique(lbl.numpy())).issubset(valid_labels):
        print("INVALID LABEL !! > ", np.unique(lbl.numpy()))
        all_valid = False
        
if all_valid:
    print("ALL VALID :D !!")

#### Training label colours

In [ ]:
training_data.label_colours()

#### Training label names

In [ ]:
training_data.label_names()

#### Mean and standard deviation of the training images

The images are normalized with the mean and the standard deviation of the images that form the training dataset.

In [ ]:
training_data.tf

In [ ]:
t_mean = []
t_std0 = []

for img, lbl in training_data:
    # shape (3, weight, height)
    numpy_image = img.numpy()
    
    # shape (3,)
    img_mean = np.mean(numpy_image, axis=(1,2)) 
    img_std0 = np.std(numpy_image, axis=(1,2))
    
    t_mean.append(img_mean)
    t_std0.append(img_std0)

# shape (N, 3) -> (mean across 0th axis) -> shape (3,)
t_mean = np.array(t_mean).mean(axis=0)
t_std0 = np.array(t_std0).mean(axis=0)


In [ ]:
print("Mean = ", t_mean)

In [ ]:
print("Std. deviation (df = N) = ", t_std0)